# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [4]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values = "NA")
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values = "NA")
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values = "NA", infer_schema_length=1000)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-3019050710.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [7]:
# Write your SQL query here

# Make sure to run the setup cell above first!
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [8]:
# Write your SQL query here
result = ctx.execute("""
  SELECT dest, COUNT(*) AS flight_count
  FROM flights
  GROUP BY dest
  ORDER BY flight_count DESC
  LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬──────────────┐
│ dest ┆ flight_count │
│ ---  ┆ ---          │
│ str  ┆ u32          │
╞══════╪══════════════╡
│ ORD  ┆ 17283        │
│ ATL  ┆ 17215        │
│ LAX  ┆ 16174        │
│ BOS  ┆ 15508        │
│ MCO  ┆ 14082        │
│ CLT  ┆ 14064        │
│ SFO  ┆ 13331        │
│ FLL  ┆ 12055        │
│ MIA  ┆ 11728        │
│ DCA  ┆ 9705         │
└──────┴──────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [9]:
# Write your SQL query here
result = ctx.execute("""
  SELECT *
  FROM flights
  WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [11]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    origin,
    AVG(dep_delay) as avg_delay
FROM flights
GROUP BY origin
""")

print(result)

shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ LGA    ┆ 10.346876 │
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
└────────┴───────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [14]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
print(result)

# Now write your query to find busiest month
result = ctx.execute("""
SELECT month, COUNT(*) AS flight_count
FROM flights
GROUP BY month
ORDER BY flight_count DESC
LIMIT 1
""")

print(result)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [20]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    carrier,
    AVG(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS ontime_rate
FROM flights
GROUP BY carrier;
""")
print(result)

shape: (16, 2)
┌─────────┬─────────────┐
│ carrier ┆ ontime_rate │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ F9      ┆ 0.715328    │
│ YV      ┆ 0.647255    │
│ OO      ┆ 0.71875     │
│ AA      ┆ 0.824376    │
│ US      ┆ 0.849873    │
│ …       ┆ …           │
│ B6      ┆ 0.76537     │
│ WN      ┆ 0.719593    │
│ AS      ┆ 0.865546    │
│ UA      ┆ 0.780431    │
│ 9E      ┆ 0.701246    │
└─────────┴─────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [24]:
# Write your SQL query here
result = ctx.execute("""
SELECT
  f.carrier,
  a.name AS airline_name,
  f.flight,
  f.origin,
  f.dest
FROM flights f
JOIN airlines a ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [25]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    AVG(2013 - p.year) AS average_plane_age
FROM flights f
JOIN planes p ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────────┐
│ carrier ┆ average_plane_age │
│ ---     ┆ ---               │
│ str     ┆ f64               │
╞═════════╪═══════════════════╡
│ FL      ┆ 11.385829         │
│ AA      ┆ 25.869426         │
│ YV      ┆ 9.313758          │
│ US      ┆ 9.103663          │
│ AS      ┆ 3.33662           │
│ …       ┆ …                 │
│ 9E      ┆ 7.101053          │
│ MQ      ┆ 35.319            │
│ UA      ┆ 13.207691         │
│ F9      ┆ 4.87874           │
│ OO      ┆ 6.84375           │
└─────────┴───────────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [28]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT
        f.year,
        f.month,
        f.day,
        f.dep_delay,
        f.origin,
        f.dest,
        w.wind_speed,
        w.precip
    FROM flights AS f
    JOIN weather AS w
        ON f.origin = w.origin
        AND f.year = w.year
        AND f.month = w.month
        AND f.day = w.day
        AND f.hour = w.hour
    WHERE f.dep_delay > 30
      AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)


shape: (4_938, 8)
┌──────┬───────┬─────┬───────────┬────────┬──────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ dep_delay ┆ origin ┆ dest ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---       ┆ ---    ┆ ---  ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64       ┆ str    ┆ str  ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪═══════════╪════════╪══════╪════════════╪════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 47        ┆ JFK    ┆ TPA  ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 1   ┆ 116       ┆ JFK    ┆ LAS  ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 39        ┆ EWR    ┆ JAX  ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 40        ┆ EWR    ┆ IAD  ┆ 24.16638   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 33        ┆ JFK    ┆ BOS  ┆ 20.71404   ┆ 0.0    │
│ …    ┆ …     ┆ …   ┆ …         ┆ …      ┆ …    ┆ …          ┆ …      │
│ 2013 ┆ 9     ┆ 22  ┆ 41        ┆ LGA    ┆ ATL  ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 9     ┆ 22  ┆ 97        ┆ LGA    ┆ PIT  ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 9     ┆ 22  ┆ 42        

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [29]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS flight_count
FROM flights f
JOIN planes p ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY flight_count DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [35]:
# Write your SQL query here

result = ctx.execute("""
    SELECT
        origin_name,
        dest_name,
        COUNT(*) AS total_flights,
        AVG(dep_delay) AS avg_dep_delay,
        100.0 * AVG(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) AS pct_delayed_over_30
    FROM (
        SELECT
            f.dep_delay,
            ao.name AS origin_name,
            ad.name AS dest_name
        FROM flights AS f
        JOIN airports AS ao ON f.origin = ao.faa
        JOIN airports AS ad ON f.dest = ad.faa
    ) AS subquery
    GROUP BY origin_name, dest_name
    ORDER BY total_flights DESC
    LIMIT 10
""")

print(result)

shape: (10, 5)
┌─────────────────────┬──────────────────────┬───────────────┬───────────────┬─────────────────────┐
│ origin_name         ┆ dest_name            ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_over_30 │
│ ---                 ┆ ---                  ┆ ---           ┆ ---           ┆ ---                 │
│ str                 ┆ str                  ┆ u32           ┆ f64           ┆ f64                 │
╞═════════════════════╪══════════════════════╪═══════════════╪═══════════════╪═════════════════════╡
│ John F Kennedy Intl ┆ Los Angeles Intl     ┆ 11262         ┆ 8.522508      ┆ 9.829515            │
│ La Guardia          ┆ Hartsfield Jackson   ┆ 10263         ┆ 11.448621     ┆ 12.247881           │
│                     ┆ Atlanta Int…         ┆               ┆               ┆                     │
│ La Guardia          ┆ Chicago Ohare Intl   ┆ 8857          ┆ 10.740758     ┆ 13.345377           │
│ John F Kennedy Intl ┆ San Francisco Intl   ┆ 8204          ┆ 11.952691    

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [37]:
# exercise 2.3 Calculate the on-time performance rate for each carrier. Consider a flight on-time if the departure delay is <= 15 minutes.
import polars as pl

total_flights_by_carrier = flights.group_by("carrier").agg(pl.count().alias("total_flights"))

on_time_flights_by_carrier = (
    flights
    .filter(pl.col("dep_delay") <= 15)
    .group_by("carrier")
    .agg(pl.count().alias("ontime_flights"))
)

ontime_performance = (
    total_flights_by_carrier
    .join(on_time_flights_by_carrier, on="carrier", how="left")
    .with_columns(
        (pl.col("ontime_flights").fill_null(0) / pl.col("total_flights")).alias("ontime_rate")
    )
    .select(["carrier", "ontime_rate"])
    .sort("ontime_rate", descending=True)
)

print(ontime_performance)

shape: (16, 2)
┌─────────┬─────────────┐
│ carrier ┆ ontime_rate │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ HA      ┆ 0.929825    │
│ AS      ┆ 0.865546    │
│ US      ┆ 0.849873    │
│ DL      ┆ 0.830742    │
│ AA      ┆ 0.824376    │
│ …       ┆ …           │
│ FL      ┆ 0.716871    │
│ F9      ┆ 0.715328    │
│ 9E      ┆ 0.701246    │
│ EV      ┆ 0.659221    │
│ YV      ┆ 0.647255    │
└─────────┴─────────────┘


/tmp/ipython-input-23752340.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  total_flights_by_carrier = flights.group_by("carrier").agg(pl.count().alias("total_flights"))
/tmp/ipython-input-23752340.py:10: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("ontime_flights"))


In [ ]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here